<a href="https://colab.research.google.com/github/tommylouistaylor/CEGE0004/blob/master/1%20-%20Week/inductive_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Inductive Learning

Goal: search for **conjunctive and m-of-n rules** using a **brute-force approach**  

Application: Not useful in practice because when data set dimesnions grow, computational complexity grows exponentially.

## The Dataset

Define a toy-dataset that consists of

1.   list of examples ($x$s): 4 Boolean features (1=TRUE, 0=FALSE)
2.   List of Boolean target values ($y$s)

In [2]:
xs = [[0, 0, 1, 0],
      [0, 1, 0, 0],
      [0, 0, 1, 1],
      [1, 0, 0, 1],
      [0, 1, 1, 0],
      [1, 1, 0, 0],
      [0, 1, 0, 1]]

ys = [0, 0, 1, 1, 0, 0, 0]

## CONJUNCTIVE RULES TASK: 
### learn the concept $f:\{0,1\}^4 \rightarrow \{0,1\}$ that has generated this dataset

## Learning Conjunctive Rules

ML to analyse datatest for patterns in a database  by identifying frequent if-then associations; 

Hypothesis Space (HP):
* conjunctive rules make up the hypothesis space
* HP is a set of all hypotheses combinations that can be returned

Conjuctive Rule Components: 
* IFS: Antecedents (the predictors) ~ an item found within the data 
* THENS: Consequent (the prediction) ~ an item found inside the combination with the antecedent
* LINKS: logical operator 'and' (∧) links the antecendents (predictors)


Example Conjuncive Rule:
* Antecedents1 (a1) ∧ Antecedents3 (a3) ∧ Antecedents4 (a4) → Consequent (y^) 
* The prediction equal to the 'and' of the first, third & forth attributes.
* Small hypotheses Space: Only 16 in $H$ from 4 literals
* Whilst problem defined by the dataset allows for $|H|=2^{2^4}=65.536$ hypothesis.



We will represent the antecedent of conjunctive rules with a list of indices. For example, to represent the rule
above we will write:

# WORKFLOW:

1) Represent antecedent(s) of conjunctive rule with a list of indices e.g. r = [0, 2, 3] 

In [3]:
r = [0, 2, 3]

2) Make Conjunctive rule readable

Remember that python is a zero-indexed language, that is 1 needs to be subtracted to each index above.

In [5]:
def make_conjunctive_rule_readable(r):
    if r:
        res = ''
        for i in r[:-1]:
            res += 'x_' + str((i+1)) + ' ∧ '

        res += 'x_' + str((r[-1]+1))
    else:
        res = '1'
    res += ' -> y'
    return res

In [ ]:
make_conjunctive_rule_readable(r)

3) Evaluate rule on an example $x$

In [14]:
def eval_conjunctive_rule(x, r):
    res = 1 # initialize rule with 1 (neutral number for multiplication)
    for i in r:
        res *= x[i] # multiply the attribute selected by r
    return res

4) Test the evaluate function by passing the rule (r) and an example (x = [0, 1, 0, 1])

In [ ]:
x = [0, 1, 0, 1]

eval_conjunctive_rule(x, r)

This rule evaluated the example as False. Can you define an example that evaluates the rule as True?

4) define the brute-force algorithm to generate all possible rules of 4 literals. Find rule that produces 0 errors on the dataset. 

In [17]:
import itertools

def generate_all_conjunctive_rules(n):
    res = []
    for size in range(n+1):
        for rule in itertools.combinations(range(n), size):
            res.append(rule)
    return res

conjunctive_rules = generate_all_conjunctive_rules(4)

5) Output list of every combination of those rules

In [ ]:
for rule in conjunctive_rules:
    print(make_conjunctive_rule_readable(rule))

6) We now want to find the rule that produces 0 errors on the dataset. To do this we define the `count_mistakes` function which takes as input the list of predictions generated by a rule `hat_ys`, and the list of target values of the dataset `ys`.

In [18]:
def count_mistakes(hat_ys, ys):
    res = 0
    for i in range(len(hat_ys)):
        if hat_ys[i] != ys[i]:
            res += 1
    return res

7) We now write a searching algorithm that iterates over all the possible rules and counts the number of mistakes each rule makes on the dataset.

In [19]:
for rule in conjunctive_rules:
    hat_ys = []
    for x in xs:
        hat_ys.append(eval_conjunctive_rule(x, rule))

    mistakes = count_mistakes(hat_ys, ys)
    print('The rule', make_conjunctive_rule_readable(rule), 'has generated', mistakes, 'mistakes')

The rule 1 -> y has generated 5 mistakes
The rule x_1 -> y has generated 2 mistakes
The rule x_2 -> y has generated 6 mistakes
The rule x_3 -> y has generated 3 mistakes
The rule x_4 -> y has generated 1 mistakes
The rule x_1 ∧ x_2 -> y has generated 3 mistakes
The rule x_1 ∧ x_3 -> y has generated 2 mistakes
The rule x_1 ∧ x_4 -> y has generated 1 mistakes
The rule x_2 ∧ x_3 -> y has generated 3 mistakes
The rule x_2 ∧ x_4 -> y has generated 3 mistakes
The rule x_3 ∧ x_4 -> y has generated 1 mistakes
The rule x_1 ∧ x_2 ∧ x_3 -> y has generated 2 mistakes
The rule x_1 ∧ x_2 ∧ x_4 -> y has generated 2 mistakes
The rule x_1 ∧ x_3 ∧ x_4 -> y has generated 2 mistakes
The rule x_2 ∧ x_3 ∧ x_4 -> y has generated 2 mistakes
The rule x_1 ∧ x_2 ∧ x_3 ∧ x_4 -> y has generated 2 mistakes


## CONCLUSION: Based on this result we can state that, since no rule generated 0 errors, the concept we want to learn does not belong to the hypothesis space defined by conjunctive rules.

## M-of-N Rules

We now choose as hypothesis space the m-of-n rules. An m-of-n rule consists of a body containing a set of $n$ literals
and a positive integer $m$. This rule is true iff $m$ of the literals are true. For example:

$\text{2-of-}\{x_1, x_2, x_3\} \rightarrow \hat{y}$

This rule is true only if at least two of the literals among $x_1$, $x_2$, and $x_3$ are true.

Note that one can prove that the hypothesis space of conjunctive rules is contained in the hypothesis space of
m-of-n rules.

Let's first define two helper methods:
1. `make_m_of_n_rule_readable` to make a m-of-n rule readable.
1. `eval_m_of_n_rule` to evaluate a m-of-n rule given an example $x$.

We will represent the body of the rule with a list containing $m$ and a list of $n$ indices. For example,
to represent the rule above we will write:

In [ ]:
r = [2, [0, 1, 2]]

The following function will make this rule readable.

In [ ]:
def make_m_of_n_rule_readable(r):
    m, variables = r
    res = str(m) + '-of-{'
    for i in variables[:-1]:
        res += 'x_' + str(i+1) + ', '
    res += 'x_' + str(variables[-1]+1) + '}'
    res += ' -> y'
    return res

In [ ]:
make_m_of_n_rule_readable(r)

The following function will evaluate the rule on an example.

In [ ]:
def eval_m_of_n_rule(x, r):
    count = 0 # initialize rule with 0 (neutral number for summation)
    m, variables = r
    for i in variables:
        count += x[i] # count how many variables match
    if count >= m:
        return 1
    return 0

To test this function let's first define an example, then test it on the rule above.

In [ ]:
x = [0, 1, 1, 1]

eval_m_of_n_rule(x, r)

This rule evaluated the example as True. Can you define an example that evaluates the rule as False?

We now define the brute-force algorithm to generate all possible m-of-n rules for 4 literals.

In [ ]:
def generate_all_m_of_n_rules(n):
    res = []
    for size in range(n+1):
        for variables in itertools.combinations(range(n), size):
            for m in range(len(variables)):
                res.append((m + 1, variables))
    return res

m_of_n_rules = generate_all_m_of_n_rules(4)

Let's now have a look at those rules.

In [ ]:
for rule in m_of_n_rules:
    print(make_m_of_n_rule_readable(rule))

These are all possible m-of-n rules one can come up with.

At this point we want to find the rule that produces 0 errors on the dataset. To do this we will reuse the
`count_mistakes` function defined above.
We now write a searching algorithm that iterates over all the possible rules and counts the number of mistakes each rule
makes on the dataset.

In [ ]:
for r in m_of_n_rules:
    hat_ys = []
    for x in xs:
        hat_ys.append(eval_m_of_n_rule(x, r))

    mistakes = count_mistakes(hat_ys, ys)
    print('The rule', make_m_of_n_rule_readable(r), 'has generated', mistakes, 'mistakes')

In this case we have found that one of the rules has generated 0 mistakes:

$\text{2-of-}\{x_1, x_3, x_4\} \rightarrow y$

Can you now say that this is the rule that generated the training set, the concept we wanted to learn?

Can you add an example in the dataset that makes also m-of-n rules fail?
Make sure to not repeat an instance of the dataset with a different target value.